**Facultad de Ciencias Exactas, Ingeniería y Agrimensura - UNR**

Tecnicatura Universitaria en Inteligencia Artificial

# Procesamiento del Lenguaje Natural - Trabajo Práctico N°: 1.

Integrantes:
- Pace, Bruno. Legajo: P-5295/7.
- Sancho Almenar, Mariano. Legajo: S-5778/9.

In [21]:
# ==========================
# Instalación de Paquetes Externos
# ==========================
!pip install sentiment_analysis_spanish
!pip install transformers
!pip install sentence-transformers

# ==========================
# Importación de Librerías
# ==========================
# Librerías básicas para manejo de datos
import pandas as pd
import warnings

# NLP y procesamiento de texto
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Modelos y herramientas de ML
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Análisis de sentimientos en español
from sentiment_analysis_spanish import sentiment_analysis

# Librerías de transformers
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

# Configuración de warnings
warnings.filterwarnings('ignore')

# Descarga de recursos adicionales de NLTK
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True


# Carga del dataset

In [22]:
df: pd.DataFrame = pd.read_csv('dataset.csv', index_col=0)

In [23]:
df.sample(5)

,Title,Description,Category,Type
1039,5/25/1977,"Alienated, hopeful-filmmaker Pat Johnson's epi...","['Comedy', 'Drama']",Pelicula
1146,The Edge of Seventeen,High-school life gets even more unbearable for...,"['Comedy', 'Drama']",Pelicula
440,Skull,Edited description from Bruno Faidutti's write...,"['Bluffing', 'Card Game', 'Party Game']",Juego
564,1775: Rebellion,The Birth of America series continues with The...,"['Age of Reason', 'American Revolutionary War'...",Juego
618,Dinosaur Island: Rawr 'n Write,Dinosaur Island: Rawr 'n Write is a roll-and-w...,"['Animals', 'Dice', 'Science Fiction']",Juego



# Funciones auxiliares

In [52]:
"""
Celda de traducción de frases
Se utiliza el modelo de traducción automática MarianMT
"""

# Define el modelo y el tokenizador
modelo = 'Helsinki-NLP/opus-mt-es-en'
tokenizer = MarianTokenizer.from_pretrained(modelo)
model = MarianMTModel.from_pretrained(modelo)

def translator(frase_usuario: str) -> str:
    """
    Traduce una frase de español a inglés.
    """
    # Tokeniza el texto y genera la traducción
    inputs = tokenizer(frase_usuario, return_tensors="pt")
    outputs = model.generate(**inputs)

    # Decodifica y muestra la traducción
    texto_ingles = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return texto_ingles

In [25]:
"""
Traductor de inglés a español
"""
modelo_traduccion_en_es = 'Helsinki-NLP/opus-mt-en-es'
tokenizer_en_es = MarianTokenizer.from_pretrained(modelo_traduccion_en_es)
model_traduccion_en_es = MarianMTModel.from_pretrained(modelo_traduccion_en_es).to("cuda")

def translator_en_to_es(text: str) -> str:

    inputs = tokenizer_en_es(text, return_tensors="pt").to("cuda")
    outputs = model_traduccion_en_es.generate(**inputs)
    texto_espanol = tokenizer_en_es.decode(outputs[0], skip_special_tokens=True)

    return texto_espanol


In [26]:
def prompt_input() -> str:
  """
  Pide al usuario que ingrese su temática a explorar.
  """
  preference_input = input("¿Que temática te interesaría abordar?: ")

  return translator(preference_input)


## Análisis de sentimientos

In [49]:
def sentiment_input() -> str:
  """
  Pide al usuario que ingrese un prompt. Retorna el prompt ingresado traducido.
  """
  sentiment_input = input("¿Cómo te sientes hoy?: ")

  return sentiment_input

In [41]:
# Cargamos el tokenizador y el modelo.
model_name: str = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer: BertTokenizer = BertTokenizer.from_pretrained(model_name)

# seteamos el modelo para utilizar gpu
model = BertForSequenceClassification.from_pretrained(model_name).to("cuda")

# Creación de pipeline.
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0)


In [61]:
sentiment = sentiment_analysis.SentimentAnalysisSpanish()

In [42]:
def get_sentiment(text: str) -> str:
    """
    Retorna el sentimiento de un texto, mediante el uso de transformers.
    """
    label: str = nlp(text)[0]['label']
    score: float = nlp(text)[0]['score']

    if label in ['5 stars', '4 stars']:
        # print(score)
        return 'Alegre'
    elif label == '3 stars':
        # print(score)
        return 'Ni fu ni fa'
    elif label in ['2 stars', '1 star']:
        # print(score)
        return 'Melancólico'


### Prueba del modelo de analisis de sentimientos

In [31]:
test_list: list[str] = [
    "Estoy muy feliz",
    "Me siento entusiasmado",
    "Estoy tranquilo",
    "Me siento relajado",
    "Estoy aburrido",
    "Me siento nostálgico",
    "Estoy un poco triste",
    "Me siento melancólico",
    "Estoy frustrado",
    "Me siento ansioso",
    "Estoy enojado",
    "Me siento decepcionado",
    "Estoy muy emocionado",
    "Me siento inspirado",
    "Estoy cansado",
    "Me siento agotado",
    "Estoy preocupado",
    "Me siento optimista",
    "Estoy motivado",
    "Me siento agradecido",
    "Estoy desconcentrado",
    "Me siento esperanzado",
    "Estoy estresado",
    "Me siento insatisfecho",
    "Estoy satisfecho",
    "Me siento apático",
    "Estoy en paz",
    "Me siento irritado",
    "Estoy un poco nervioso",
    "Me siento confiado",
    "Estoy renovado",
    "Me siento pleno",
    "Estoy resignado",
    "Me siento inquieto",
    "Estoy contento",
    "Me siento vulnerable",
    "Estoy enamorado",
    "Me siento frustrado",
    "Estoy ilusionado",
    "Me siento reflexivo",
    "Estoy agradecido",
    "Me siento emocionalmente agotado"
]

for text in test_list:
  print(f'{text} -> {get_sentiment(text)}')

Estoy muy feliz -> Alegre
Me siento entusiasmado -> Alegre
Estoy tranquilo -> Alegre
Me siento relajado -> Melancólico
Estoy aburrido -> Melancólico
Me siento nostálgico -> Alegre
Estoy un poco triste -> Ni fu ni fa
Me siento melancólico -> Melancólico
Estoy frustrado -> Melancólico
Me siento ansioso -> Alegre
Estoy enojado -> Melancólico
Me siento decepcionado -> Melancólico
Estoy muy emocionado -> Alegre
Me siento inspirado -> Alegre
Estoy cansado -> Melancólico
Me siento agotado -> Melancólico
Estoy preocupado -> Melancólico
Me siento optimista -> Ni fu ni fa
Estoy motivado -> Alegre
Me siento agradecido -> Alegre
Estoy desconcentrado -> Melancólico
Me siento esperanzado -> Alegre
Estoy estresado -> Melancólico
Me siento insatisfecho -> Melancólico
Estoy satisfecho -> Alegre
Me siento apático -> Melancólico
Estoy en paz -> Alegre
Me siento irritado -> Melancólico
Estoy un poco nervioso -> Ni fu ni fa
Me siento confiado -> Alegre
Estoy renovado -> Alegre
Me siento pleno -> Ni fu ni f

A fin de realizar una comparación, utilizamos sentiment-analysis sobre la misma lista de test.

El rendimiento obtenido es inferior, teniendo en cuenta que un resultado cercano a uno es un texto positivo, cercano a cero es negativo y en los valores medios es neutro. Por eso se optó por los transformers.

Para obtener esta conclusión se visitó la [Documentación](https://pypi.org/project/sentiment-analysis-spanish/).

In [62]:
for texto in test_list:
  print(f'{texto} -> {sentiment.sentiment(texto)}')

Estoy muy feliz -> 0.6889487711193526
Me siento entusiasmado -> 0.0728752984896165
Estoy tranquilo -> 0.7826378949098491
Me siento relajado -> 0.23951579470116582
Estoy aburrido -> 0.31062133515766494
Me siento nostálgico -> 0.0728752984896165
Estoy un poco triste -> 0.04078651378014973
Me siento melancólico -> 0.0728752984896165
Estoy frustrado -> 0.31062133515766494
Me siento ansioso -> 0.0728752984896165
Estoy enojado -> 0.31062133515766494
Me siento decepcionado -> 0.005644903897282337
Estoy muy emocionado -> 0.537734459784737
Me siento inspirado -> 0.0728752984896165
Estoy cansado -> 0.31062133515766494
Me siento agotado -> 0.0728752984896165
Estoy preocupado -> 0.31062133515766494
Me siento optimista -> 0.0728752984896165
Estoy motivado -> 0.31062133515766494
Me siento agradecido -> 0.0728752984896165
Estoy desconcentrado -> 0.31062133515766494
Me siento esperanzado -> 0.0728752984896165
Estoy estresado -> 0.31062133515766494
Me siento insatisfecho -> 0.0728752984896165
Estoy sat


### Analisis de sentimiento del contenido a partir de la columna 'Category'

A partir de las categorías/generos que abarca el contenido, se clasificará para que lo recomiende dependiendo los sentimientos

In [33]:
df_sentiment: pd.DataFrame = df.copy()

In [34]:
df_sentiment['Sentiment'] = df['Category'].apply(get_sentiment)

In [35]:
df_sentiment[['Title', 'Category', 'Sentiment']].sample(5)

,Title,Category,Sentiment
664,Faraway,"['Adventure', 'Card Game', 'Fantasy']",Alegre
1564,Snow White and the Huntsman,"['Action', 'Adventure', 'Drama']",Alegre
2160,On the Origin of Species By Means of Natural S...,['Evolution (Biology)'],Alegre
402,Mindbug: First Contact,"['Card Game', 'Fantasy']",Alegre
2571,"The Philippine Islands, 1493-1898 — Volume 07 ...",['Philippines -- History -- Sources'],Alegre



## Obtención de recomendaciones

Se utiliza SentenceTransformer usando el modelo preentrenado ‘msmarco-MiniLM-L-6-v3’ para calcular las recomendaciones de contenido

In [43]:
modelo_recomendacion: SentenceTransformer = SentenceTransformer('msmarco-MiniLM-L-6-v3', device='cuda')
incrustaciones_respuestas = modelo_recomendacion.encode(df_sentiment['Description'])

In [44]:
def get_prompt_recommendation(sentiment: str, preference: str) -> list[str]:
  """
  Recibe el sentimiento y el prompt de preferencia, devuelve una lista de strings con las recomendaciones.
  """
  df_sentiment_filtered = df_sentiment[df_sentiment['Sentiment'] == sentiment]
  df_sentiment_filtered = df_sentiment_filtered.reset_index()

  #print(df_sentiment_filtered)
  incrustaciones_respuestas = modelo_recomendacion.encode(df_sentiment_filtered['Description'])
  incrustaciones_consultas = modelo_recomendacion.encode(preference)

  similitudes = util.cos_sim(incrustaciones_consultas, incrustaciones_respuestas)[0]
  mejor_indice = similitudes.argmax().item()  # Convertimos a entero

  categoria_traducida = translator_en_to_es(df_sentiment_filtered['Category'].iloc[mejor_indice])
  titulo_traducido = translator_en_to_es(df_sentiment_filtered['Title'].iloc[mejor_indice])
  tipo = translator_en_to_es(df_sentiment_filtered['Type'].iloc[mejor_indice])
  descripcion_traducida = translator_en_to_es(df_sentiment_filtered['Description'].iloc[mejor_indice])

  return titulo_traducido, descripcion_traducida, tipo, categoria_traducida


# Código programa

In [59]:
def principal_function() -> None:
    """
    Función principal del programa. Permite al usuario ingresar un sentimiento y un prompt,
    y muestra las recomendaciones correspondientes hasta que se presiona la tecla 'q' para salir.
    """
    print('¡Bienvenido al recomendador de actividades! \n')

    while True:
        user_input = input('¿Desea continuar? (Presione "N" para salir o cualquier tecla para continuar): ')
        if user_input.lower() == 'n':
            print("Saliendo del programa...")
            break

        print(f'Continuaremos con su recomendación: \n')

        sentimiento = sentiment_input()
        sentimiento = get_sentiment(sentimiento)

        print(f'Sentimiento: {sentimiento}')

        prompt = prompt_input()

        titulo, descripcion, tipo, categoria = get_prompt_recommendation(sentimiento, prompt)

        print(f'La recomendación es: {titulo}')
        print(f'Tipo: {tipo}')
        print(f'Categorías: {categoria}')
        print(f'Trama: {descripcion}')
        print('---------------------------------------------------')



In [63]:
principal_function()

¡Bienvenido al recomendador de actividades! 

¿Desea continuar? (Presione "N" para salir o cualquier tecla para continuar): Si
Continuaremos con su recomendación: 

¿Cómo te sientes hoy?: Feliz
Sentimiento: Alegre
¿Que temática te interesaría abordar?: Grupo de amigos
La recomendación es: El fin del mundo
Tipo: Pelicula
Categorías: ['Acción', 'Comedia', 'Ci-Fi']
Trama: Cinco amigos que se reúnen en un intento de superar su épica red de pubs de veinte años antes sin saberlo se convierten en la única esperanza de supervivencia de la humanidad.
---------------------------------------------------
¿Desea continuar? (Presione "N" para salir o cualquier tecla para continuar): N
Saliendo del programa...
